In [1]:
import requests
import pandas as pd

In [2]:
# Substitua "{API key}" pela sua chave de API real
api_key = "550d57361bde0d30b0dd389950c781ad"

# Latitude e longitude fictícias
latitude = 33.44
longitude = -94.04

# Parâmetros da chamada de API
params = {
    "lat": latitude,
    "lon": longitude,
    "exclude": "hourly,daily",
    "appid": api_key
}

# URL da API
url = "https://api.openweathermap.org/data/3.0/onecall"

# Fazendo a chamada da API
response = requests.get(url, params=params)

# Verificando se a chamada foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    # Aqui você pode manipular os dados da resposta conforme necessário
    print(data)
else:
    print("A chamada da API falhou. Código de status:", response.status_code)


A chamada da API falhou. Código de status: 401


In [6]:
link2 = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [3]:
link2

'https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad'

In [7]:
requisicao = requests.get(link2)

In [8]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 285.97,
  'feels_like': 285.55,
  'temp_min': 285.37,
  'temp_max': 286.94,
  'pressure': 1021,
  'humidity': 86,
  'sea_level': 1021,
  'grnd_level': 1016},
 'visibility': 10000,
 'wind': {'speed': 0.97, 'deg': 144, 'gust': 2.3},
 'clouds': {'all': 97},
 'dt': 1697177728,
 'sys': {'type': 2,
  'id': 2020383,
  'country': 'BR',
  'sunrise': 1697186946,
  'sunset': 1697232733},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [5]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 289.9,
  'feels_like': 290.08,
  'temp_min': 289.66,
  'temp_max': 291.34,
  'pressure': 1009,
  'humidity': 94,
  'sea_level': 1009,
  'grnd_level': 1005},
 'visibility': 9549,
 'wind': {'speed': 0.98, 'deg': 212, 'gust': 3.5},
 'clouds': {'all': 100},
 'dt': 1696462388,
 'sys': {'type': 2,
  'id': 2020383,
  'country': 'BR',
  'sunrise': 1696409976,
  'sunset': 1696454810},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}